In [1]:
exp_name = "lstm3000"

## Only RNN

In [2]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pickle 

import sys
sys.path.append('../src')
sys.path.append('../src/models')
from lstm_model import *
from training import *
from nn_eval import *

## Loading synthetic data

In [3]:
#loading data
data_path= '/data2/data/zebrahim/synthetic_dataset/data_8192_1000_3_4_4_.03_.03_.2' + 'processed_data.p'

with open(data_path, 'r') as fin:
    data = pickle.load(fin)
    
train_data = data['train_data']
validation_data = data['validation_data']
test_data = data['test_data']

train_gt = np.expand_dims(data['train_gt'], axis=-1)
validation_gt = np.expand_dims(data['validation_gt'], axis=-1)
test_gt = np.expand_dims(data['test_gt'], axis=-1)

index_of_changes = data['index_of_changes']

## Model

In [4]:
## Model Parameter
n_variables= 12
learning_rate = 0.001
batch_size = 100

lstm_n_hidden = 128
n_classes = 1

#positive weight for weighted cross entropy
pos_weight = (1.0 - train_gt.mean()) / train_gt.mean()

In [5]:
pos_weight

3.3908369237096849

In [6]:
model = Model(n_variables = 12,
              n_classes = 1)

In [7]:
# Launch the graph
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)             

In [8]:
summary_path = '../summary/' + exp_name 
checkpoint_path = '../model/' + exp_name + '/' + exp_name 

In [9]:
model.build_lstm_net(lstm_n_hidden,
                     n_classes,
                     learning_rate,
                    pos_weight)

In [ ]:
train_obj = Trainer(model,
                 train_data, train_gt,
                 validation_data, validation_gt,
                 sess,
                 summary_path,
                 batch_size,
                 global_step = 0)

In [ ]:
train_obj.train(max_iter = 1000,
                train_eval_step=10,
                validation_eval_step=10,
                display_step=10)


Minibatch Loss= 1.327543
Tarin Iter 0, auc= 0.243885, fmeasure_max= 0.380282, precision= 0.234788, recall= 0.999901
Train Random guess:  , auc= 0.233734fmeasure_max= 0.378786, precision= 0.233644, recall= 0.999990
Validation Iter 0, auc= 0.196965, fmeasure_max= 0.356383, precision= 0.216844, recall= 0.999662
Validation Random guess:  auc= 0.216343, fmeasure_max= 0.356357, precision= 0.216809, recall= 1.000000
Minibatch Loss= 1.106681
Tarin Iter 10, auc= 0.220893, fmeasure_max= 0.372123, precision= 0.246349, recall= 0.760291
Train Random guess:  , auc= 0.219121fmeasure_max= 0.357776, precision= 0.217866, recall= 0.999877
Validation Iter 10, auc= 0.199565, fmeasure_max= 0.367652, precision= 0.233874, recall= 0.859017
Validation Random guess:  auc= 0.215329, fmeasure_max= 0.356434, precision= 0.216885, recall= 0.999606


In [ ]:
saver = tf.train.Saver()
saver.save(sess, checkpoint_path, global_step=train_obj.global_step)

In [ ]:
saver = tf.train.Saver()
saver.restore(sess, checkpoint_path+'-1000')

In [ ]:
evaluate_model(model,
               sess,
               test_data,
               test_gt)